In [6]:
import pandas as pd
import requests
import time
from urllib.parse import quote

# Read schools.csv
df_schools = pd.read_csv('schools.csv')

print(f"Total schools: {len(df_schools)}")
print(f"\nColumns in schools.csv: {', '.join(df_schools.columns.tolist())}")
print(f"\nFirst few rows:")
print(df_schools[['school_name', 'address', 'postal_code']].head())

# Prepare results list
results = []

# OneMap API endpoint
onemap_base_url = "https://www.onemap.gov.sg/api/common/elastic/search"

# Headers with authentication token
headers = {
    "Authorization": f"Bearer {API_TOKEN}",
    "accept": "application/json"
}

print(f"\n\nFetching coordinates for each school...")
print(f"Progress will be shown every 50 schools\n")

# Process each school
for idx, row in df_schools.iterrows():
    school_name = row['school_name']
    address = row['address']
    postal_code = str(row['postal_code']).strip()
    
    # Get additional columns from original CSV
    mainlevel_code = row.get('mainlevel_code', None)
    type_code = row.get('type_code', None)
    zone_code = row.get('zone_code', None)
    
    # Skip if postal code is missing or invalid
    if pd.isna(postal_code) or postal_code == 'nan' or postal_code == '':
        print(f"[{idx+1}/{len(df_schools)}] Skipping {school_name}: No postal code")
        results.append({
            'school_name': school_name,
            'address': address,
            'postal_code': postal_code,
            'mainlevel_code': mainlevel_code,
            'type_code': type_code,
            'zone_code': zone_code,
            'X': None,
            'Y': None,
            'Longitude': None,
            'Latitude': None
        })
        continue
    
    # Prepare search value: combine address and postal code
    # Clean address (remove extra spaces) and combine with postal code
    clean_address = str(address).strip() if not pd.isna(address) else ''
    search_value = f"{clean_address} {postal_code}".strip()
    
    # URL encode the search value to handle special characters
    encoded_search = quote(search_value)
    
    # Make API call with combined address and postal code
    search_url = f"{onemap_base_url}?searchVal={encoded_search}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    
    try:
        response = requests.get(search_url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            
            # Get first result
            if 'results' in data and len(data['results']) > 0:
                first_result = data['results'][0]
                
                # Extract coordinates
                x = first_result.get('X', None)
                y = first_result.get('Y', None)
                longitude = first_result.get('LONGITUDE', None)
                latitude = first_result.get('LATITUDE', None)
                
                results.append({
                    'school_name': school_name,
                    'address': address,
                    'postal_code': postal_code,
                    'mainlevel_code': mainlevel_code,
                    'type_code': type_code,
                    'zone_code': zone_code,
                    'X': x,
                    'Y': y,
                    'Longitude': longitude,
                    'Latitude': latitude
                })
                
                if (idx + 1) % 50 == 0:
                    print(f"[{idx+1}/{len(df_schools)}] Processed {school_name}")
            else:
                print(f"[{idx+1}/{len(df_schools)}] No results for {school_name} (search: {search_value})")
                results.append({
                    'school_name': school_name,
                    'address': address,
                    'postal_code': postal_code,
                    'mainlevel_code': mainlevel_code,
                    'type_code': type_code,
                    'zone_code': zone_code,
                    'X': None,
                    'Y': None,
                    'Longitude': None,
                    'Latitude': None
                })
        else:
            print(f"[{idx+1}/{len(df_schools)}] API error for {school_name}: Status {response.status_code}")
            results.append({
                'school_name': school_name,
                'address': address,
                'postal_code': postal_code,
                'mainlevel_code': mainlevel_code,
                'type_code': type_code,
                'zone_code': zone_code,
                'X': None,
                'Y': None,
                'Longitude': None,
                'Latitude': None
            })
        
        # Small delay to avoid rate limiting
        time.sleep(0.1)
        
    except Exception as e:
        print(f"[{idx+1}/{len(df_schools)}] Error processing {school_name}: {e}")
        results.append({
            'school_name': school_name,
            'address': address,
            'postal_code': postal_code,
            'mainlevel_code': mainlevel_code,
            'type_code': type_code,
            'zone_code': zone_code,
            'X': None,
            'Y': None,
            'Longitude': None,
            'Latitude': None
        })

# Create DataFrame from results
df_results = pd.DataFrame(results)

print(f"\n\n✓ API calls completed!")
print(f"\nTotal schools processed: {len(df_results)}")
print(f"Schools with coordinates: {df_results['X'].notna().sum()}")
print(f"Schools without coordinates: {df_results['X'].isna().sum()}")

print(f"\n\nSummary of results:")
print(f"  - Successful API calls: {df_results['X'].notna().sum()}")
print(f"  - Failed/No results: {df_results['X'].isna().sum()}")
print(f"  - API errors: Check output above for details")

print(f"\n\nFirst 10 rows with coordinates:")
print(df_results[df_results['X'].notna()].head(10).to_string(index=False))

print(f"\n\nFirst 5 rows without coordinates (if any):")
missing = df_results[df_results['X'].isna()]
if len(missing) > 0:
    print(missing.head(5)[['school_name', 'postal_code']].to_string(index=False))
else:
    print("  All schools have coordinates!")


Total schools: 337

Columns in schools.csv: school_name, url_address, address, postal_code, telephone_no, telephone_no_2, fax_no, fax_no_2, email_address, mrt_desc, bus_desc, principal_name, first_vp_name, second_vp_name, third_vp_name, fourth_vp_name, fifth_vp_name, sixth_vp_name, dgp_code, zone_code, type_code, nature_code, session_code, mainlevel_code, sap_ind, autonomous_ind, gifted_ind, ip_ind, mothertongue1_code, mothertongue2_code, mothertongue3_code

First few rows:
                      school_name                   address  postal_code
0        ADMIRALTY PRIMARY SCHOOL    11 WOODLANDS CIRCLE          738907
1      ADMIRALTY SECONDARY SCHOOL  31 WOODLANDS CRESCENT          737916
2    AHMAD IBRAHIM PRIMARY SCHOOL    10 YISHUN STREET 11          768643
3  AHMAD IBRAHIM SECONDARY SCHOOL    751 YISHUN AVENUE 7          768928
4                  AI TONG SCHOOL  100 Bright Hill Drive          579646


Fetching coordinates for each school...
Progress will be shown every 50 schools



In [7]:
# Manually add coordinates for SCHOOL OF THE ARTS, SINGAPORE
# This school didn't return results from the API

try:
    # Check if df_results exists
    if 'df_results' not in globals():
        print("Error: df_results not found. Please run the previous cell first to fetch API data.")
    else:
        from pyproj import Transformer
        
        # Create transformer for WGS84 to SVY21 conversion
        transformer = Transformer.from_crs("EPSG:4326", "EPSG:3414", always_xy=True)
        
        # Manual coordinates for SCHOOL OF THE ARTS, SINGAPORE
        school_name = "SCHOOL OF THE ARTS, SINGAPORE"
        postal_code = "227968"
        longitude = 103.8490353
        latitude = 1.2990858
        
        # Convert to SVY21
        x, y = transformer.transform(longitude, latitude)
        
        # Find and update the row
        mask = (df_results['school_name'] == school_name) & (df_results['postal_code'] == postal_code)
        
        if mask.any():
            # Update the row
            df_results.loc[mask, 'Longitude'] = longitude
            df_results.loc[mask, 'Latitude'] = latitude
            df_results.loc[mask, 'X'] = x
            df_results.loc[mask, 'Y'] = y
            
            print(f"✓ Updated coordinates for: {school_name}")
            print(f"  Postal Code: {postal_code}")
            print(f"  Longitude: {longitude}")
            print(f"  Latitude: {latitude}")
            print(f"  SVY21 X: {x:.6f}")
            print(f"  SVY21 Y: {y:.6f}")
            
            # Show the updated row
            print(f"\nUpdated row:")
            print(df_results[mask][['school_name', 'postal_code', 'Longitude', 'Latitude', 'X', 'Y']].to_string(index=False))
            
            print(f"\n\nUpdated summary:")
            print(f"  Total schools: {len(df_results)}")
            print(f"  Schools with coordinates: {df_results['X'].notna().sum()}")
            print(f"  Schools without coordinates: {df_results['X'].isna().sum()}")
        else:
            print(f"Warning: Could not find school '{school_name}' with postal code '{postal_code}'")
            print(f"Available schools with missing coordinates:")
            missing = df_results[df_results['X'].isna()]
            if len(missing) > 0:
                print(missing[['school_name', 'postal_code']].to_string(index=False))
            else:
                print("  No schools with missing coordinates found")
                
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pyproj"])
    from pyproj import Transformer
    print("Installed pyproj. Please run this cell again.")
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()


✓ Updated coordinates for: SCHOOL OF THE ARTS, SINGAPORE
  Postal Code: 227968
  Longitude: 103.8490353
  Latitude: 1.2990858
  SVY21 X: 29749.130693
  SVY21 Y: 31271.831428

Updated row:
                  school_name postal_code   Longitude  Latitude             X             Y
SCHOOL OF THE ARTS, SINGAPORE      227968  103.849035  1.299086  29749.130693  31271.831428


Updated summary:
  Total schools: 337
  Schools with coordinates: 337
  Schools without coordinates: 0


In [8]:
# Save results to CSV file
# This cell writes the df_results DataFrame (created in previous cell) to schools_coord.csv

try:
    # Check if df_results exists
    if 'df_results' not in globals():
        print("Error: df_results not found. Please run the previous cell first to fetch API data.")
    else:
        # Save to CSV
        df_results.to_csv('schools_coord.csv', index=False)
        
        print(f"✓ CSV file saved as 'schools_coord.csv'")
        print(f"\nFile summary:")
        print(f"  Total rows: {len(df_results)}")
        print(f"  Columns: {', '.join(df_results.columns.tolist())}")
        print(f"  Schools with coordinates: {df_results['X'].notna().sum()}")
        print(f"  Schools without coordinates: {df_results['X'].isna().sum()}")
        
        print(f"\n\nSample of saved data (first 5 rows):")
        print(df_results.head().to_string(index=False))
        
except Exception as e:
    print(f"Error saving CSV: {e}")
    import traceback
    traceback.print_exc()


✓ CSV file saved as 'schools_coord.csv'

File summary:
  Total rows: 337
  Columns: school_name, address, postal_code, mainlevel_code, type_code, zone_code, X, Y, Longitude, Latitude
  Schools with coordinates: 337
  Schools without coordinates: 0


Sample of saved data (first 5 rows):
                   school_name                  address postal_code    mainlevel_code            type_code zone_code                X                Y        Longitude         Latitude
      ADMIRALTY PRIMARY SCHOOL   11 WOODLANDS CIRCLE         738907           PRIMARY    GOVERNMENT SCHOOL     NORTH 24296.6250855923 47144.7700594738 103.800040119743  1.4426347903311
    ADMIRALTY SECONDARY SCHOOL 31 WOODLANDS CRESCENT         737916 SECONDARY (S1-S5)    GOVERNMENT SCHOOL     NORTH 24559.0472937012 47504.7873554379 103.802398196596 1.44589068910993
  AHMAD IBRAHIM PRIMARY SCHOOL   10 YISHUN STREET 11         768643           PRIMARY    GOVERNMENT SCHOOL     NORTH 27958.1371433474 46096.2627680488 103.832